In [1]:
# Imports
import pyomo.environ as pyomo
import numpy as np
import time

In [2]:
# Reading problem instance from file 
def read_problem_instance(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Data Types
    # Parse data
    num_facilities = int(lines[1].split(': ')[1])
    num_customers = int(lines[2].split(': ')[1])
    # grid_size = int(lines[3].split(': ')[1].split('x')[0])

    
    facilities_coords = []
    customers_coords = []
    distances = np.zeros((num_facilities, num_customers))
    fixed_costs = np.zeros((num_facilities), dtype=int)
    facility_capacities = []
    customer_weights = []  
    cost_s = np.zeros((num_facilities, num_customers))  
    cost_l = np.zeros((num_facilities, num_customers))  

    line_idx = 5  

    # Facilities coordinates
    for _ in range(num_facilities):
        x, y = map(int, lines[line_idx].split(','))
        facilities_coords.append((x, y))
        line_idx += 1

    # Customers coordinates
    line_idx += 1
    for _ in range(num_customers):
        x, y = map(int, lines[line_idx].split(','))
        customers_coords.append((x, y))
        line_idx += 1

    # Distances
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            distances[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Fixed costs for facility openings
    line_idx += 1
    for i in range(num_facilities):
        fixed_costs[i] = int(lines[line_idx].split(': ')[1])
        line_idx += 1

    # Drone operational costs (small drone)
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            cost_s[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Drone operational costs (large drone)
    line_idx += 1
    for i in range(num_facilities):
        for j in range(num_customers):
            cost_l[i][j] = float(lines[line_idx].split(': ')[1])
            line_idx += 1

    # Facility capacities
    line_idx += 1
    for _ in range(num_facilities):
        facility_capacities.append(int(lines[line_idx].split(': ')[1]))
        line_idx += 1

    # Small and large drone capacities and ranges
    payload_small = int(lines[line_idx].split(': ')[1])
    payload_large = int(lines[line_idx + 1].split(': ')[1])
    range_small = int(lines[line_idx + 2].split(': ')[1])
    range_large = int(lines[line_idx + 3].split(': ')[1])

    # Check if customer weights section exists
    if line_idx + 4 < len(lines) and lines[line_idx + 4].startswith("Package weights"):
        # Read customer weights if the section exists
        line_idx += 5  # Skip to the section after the drone capacities and ranges
        for _ in range(num_customers):
            customer_weight = int(lines[line_idx].split(': ')[1])
            customer_weights.append(customer_weight)
            line_idx += 1
    else:
        # If no package weights section, assign default weights or handle error
        print("Warning: No customer weights found. Using default weights.")
        customer_weights = [1] * num_customers  # Assign default weight of 1

    Ds = int(lines[line_idx].split(': ')[1])
    Dl = int(lines[line_idx + 1].split(': ')[1])

    return {
        "num_facilities": num_facilities,
        "num_customers": num_customers,
        "distances": distances,
        "fixed_costs" : fixed_costs,
        "facility_capacities": facility_capacities,
        "payload_small": payload_small,
        "payload_large": payload_large,
        "range_small": range_small,
        "range_large": range_large,
        "customer_weights": customer_weights,
        "cost_s": cost_s,
        "cost_l": cost_l,
        "Ds" : Ds,
        "Dl" : Dl
    }

In [3]:
#  TESTING  # 
data = read_problem_instance("generated_problems/class_1/problem_instance_1.txt")

data["range_large"]
data["Ds"]
data["Dl"]
data['payload_small']

###############


6

In [4]:
# Model initialization and solve
# Initialize the model
def solve_facility_problem(file_path):
    
    model = pyomo.ConcreteModel()
    
    # Reading problem file 
    data = read_problem_instance(file_path)

    # Define Sets
    F = range(data['num_facilities']) # Facilities Set
    C = range(data['num_customers']) # Customers Set
    Ds = range(data["Ds"])  # S-type Drones Set
    Dl = range(data["Dl"])  # L-type Drones Set

    # Parameters
    # Distance between facility i and customer j 
    d = data['distances']
    # Fixed cost of operating facility i
    f = data['fixed_costs']
    # Operational cost for serving customer j from facility i with an S-type  drone
    cs = data['cost_s']
    # Operational cost for serving customer j from facility i with an L-type drone
    cl = data['cost_l']
    # The capacity of each facility, the maximum number of drones it can support
    K = data['facility_capacities']
      # Weight of customer’s j package
    W = data['customer_weights']
    # Payload capacity of S-type drones
    Ps = data['payload_small']
    # Payload capacity of L-type drones
    Pl = data['payload_large']
    # Max Range of S-type drones
    Rs = data['range_small']
    # Max Range of L-type drones
    Rl = data['range_large']
  
    # Weight factor for S-type drones for the facilities
    ws = 1
    # Weight factor for L-type drones for the facilities
    wl = 2
    
    # Decision Variables 
    # Binary variable. Facility i is opened (1) or closed (0)
    model.x = pyomo.Var(F, domain=pyomo.Binary)
    # Binary variable. If customer j is served by facility i using an S-type drone (1), otherwise (0)
    model.ys = pyomo.Var(Ds, F, C, domain=pyomo.Binary)
    # Binary variable. If customer j is served by facility i using an L-type drone (1), otherwise (0)
    model.yl = pyomo.Var(Dl, F, C, domain=pyomo.Binary)

    # Define Objective Function 
    def objective_function_rule(model):
        penalty_large_drone = 0.5
        fixed_costs = sum(f[i] * model.x[i] for i in F)
        variable_costs = sum(cs[i][j] * model.ys[s, i, j] + (cl[i][j] + penalty_large_drone) * model.yl[l, i, j] for s in Ds for l in Dl for i in F for j in C)
        return fixed_costs + variable_costs

    model.objective = pyomo.Objective(rule=objective_function_rule, sense=pyomo.minimize)

    # Constraints
    # 1. Each customer must be served by one facility and one type of drone.
    def is_customer_served(model, j):
        return sum(model.ys[s, i, j] for s in Ds for i in F) + sum(model.yl[l, i, j] for l in Dl for i in F) == 1

    model.customer_served = pyomo.Constraint(C, rule=is_customer_served)

    # 2. Each facility has a total capacity limit for the number of drones it can support.
    def is_facility_maxed(model, i):
        return sum(ws * model.ys[s, i, j] for s in Ds for j in C) + sum(wl * model.yl[l, i, j] for l in Dl for j in C) <= K[i] * model.x[i]

    model.facility_capacity = pyomo.Constraint(F, rule=is_facility_maxed)

    # 3. A customer can only be assigned to a facility that is open. 
    def is_small_facility_open(model, s, i, j):
        return model.ys[s, i, j] <= model.x[i]

    model.small_facility_open = pyomo.Constraint(Ds, F, C, rule=is_small_facility_open)

    def is_large_facility_open(model, l, i, j):
        return model.yl[l, i, j] <= model.x[i]

    model.large_facility_open = pyomo.Constraint(Dl, F, C, rule=is_large_facility_open)

    # 4. The assigned drone type must be able to carry the package weight.
    def payload_capacity_small_rule(model, s, i, j):
        return W[j] * model.ys[s, i, j] <= Ps

    model.payload_capacity_small = pyomo.Constraint(Ds, F, C, rule=payload_capacity_small_rule)

    def payload_capacity_large_rule(model, l, i, j):
        return W[j] * model.yl[l, i, j] <= Pl

    model.payload_capacity_large = pyomo.Constraint(Dl, F, C, rule=payload_capacity_large_rule)

    # 5. S-type drones can only serve customers within Rs while L-type drones can serve customers within Rl
    def range_limit_small_rule(model, s, i, j):
        return d[i][j] * model.ys[s, i, j] <= Rs

    model.range_limit_small = pyomo.Constraint(Ds, F, C, rule=range_limit_small_rule)

    def range_limit_large_rule(model, l, i, j):
        return d[i][j] * model.yl[l, i, j] <= Rl

    model.range_limit_large = pyomo.Constraint(Dl, F, C, rule=range_limit_large_rule)

    # 6. Each used drone should be assigned to exactly one facility (Check this constraint again)
    def distinct_facility_assignment_small_rule(model, s):
        return sum(model.ys[s, i, j] for i in F for j in C) <= 1

    model.distinct_facility_assignment_small = pyomo.Constraint(Ds, rule=distinct_facility_assignment_small_rule)

    def distinct_facility_assignment_large_rule(model, l):
        return sum(model.yl[l, i, j] for i in F for j in C) <= 1

    model.distinct_facility_assignment_large = pyomo.Constraint(Dl, rule=distinct_facility_assignment_large_rule)


    # Solve the model
    solver_start_time = time.time()

    solver = pyomo.SolverFactory('gurobi')
    result = solver.solve(model, tee=False)
    
    solver_stop_time = time.time()
    solver_elapsed = solver_stop_time - solver_start_time

    # Check if the model is infeasible
    if result.solver.termination_condition == pyomo.TerminationCondition.infeasible:
        return result, solver_elapsed  

    # Print Results if the model is feasible
    print("-----Facility Openings-----")
    for i in F:
        print(f"Facility {i}: {'Open' if model.x[i]() > 0.5 else 'Closed'}")

    print("-----Customer Assignments-----")
    for j in C:
        assigned = False
        for i in F:
            for s in Ds:
                if model.ys[s, i, j]() > 0.5:
                    print(f"Customer {j} is served by Facility {i} using Small Drone {s}.")
                    assigned = True
            for l in Dl:
                if model.yl[l, i, j]() > 0.5:
                    print(f"Customer {j} is served by Facility {i} using Large Drone {l}.")
                    assigned = True
        if not assigned:
            print(f"Customer {j} is not assigned to any facility.")

    print(f"Total Cost: {model.objective()}")

    return result, solver_elapsed

In [5]:
with open("solve_times.txt", "w") as time_file:
    for c in range(4, 10):
        for i in range(2):
            problem_file = f"generated_problems/class_{c+1}/problem_instance_{i+1}.txt"
            print(f"Class {c+1}, Problem {i+1}\n")

            # Record starting time
            start_time = time.time()

            try:
                # Attempt to solve the facility problem
                result, solver_time = solve_facility_problem(problem_file)

                # Check if result is infeasible or error occurred
                if result.solver.status == pyomo.TerminationCondition.error:
                    print(f"Problem {i+1} encountered an error. Skipping it.\n")
                elif result.solver.termination_condition == pyomo.TerminationCondition.infeasible:
                    print(f"Problem {i+1} is infeasible, skipping it.\n")
                else:
                    # Continue processing if the result is valid
                    print(f"Problem {i+1} solved successfully.\n")
                
                    # print(f"Solver termination condition: {result.solver.termination_condition}")
                    # print(f"Solver message: {result.solver.message}")

            except Exception as e:
                # If an error occurs (e.g., solver fails), print the error and continue with the next problem
                print(f"Error solving problem {i+1}: {e}\n")

            # Record the end time and calculate the elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"Problem {i+1} Time:")
            print(f"Total Time: {elapsed_time:.2f} seconds.")
            print(f"Solver time: {solver_time:.2f} seconds.\n")

            # Save the times to the file
            time_file.write(f"Class {c+1}, Problem {i+1}:\n")
            time_file.write(f"Total Time: {elapsed_time:.2f} seconds\n")
            time_file.write(f"Solver Time: {solver_time:.2f} seconds\n\n")


Class 5, Problem 1

-----Facility Openings-----
Facility 0: Closed
Facility 1: Closed
Facility 2: Closed
Facility 3: Open
Facility 4: Closed
Facility 5: Closed
Facility 6: Closed
Facility 7: Closed
Facility 8: Closed
Facility 9: Open
Facility 10: Closed
Facility 11: Closed
Facility 12: Closed
Facility 13: Closed
Facility 14: Closed
Facility 15: Closed
Facility 16: Closed
Facility 17: Closed
Facility 18: Closed
Facility 19: Closed
Facility 20: Closed
Facility 21: Closed
Facility 22: Closed
Facility 23: Closed
Facility 24: Closed
Facility 25: Closed
Facility 26: Open
Facility 27: Closed
Facility 28: Closed
Facility 29: Closed
Facility 30: Open
Facility 31: Closed
Facility 32: Closed
Facility 33: Closed
Facility 34: Closed
Facility 35: Closed
Facility 36: Closed
Facility 37: Closed
Facility 38: Closed
Facility 39: Closed
Facility 40: Closed
Facility 41: Closed
Facility 42: Closed
Facility 43: Closed
Facility 44: Closed
-----Customer Assignments-----
Customer 0 is served by Facility 3 usin

KeyboardInterrupt: 